In [3]:
import cv2
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 2)
    
    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        x = x.view(-1, 32 * 8 * 8)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        return x

# Define the dataset and data loader
train_folder = 'C:/Users/lolol/OneDrive/Документы/Дипломная работа/train'
test_folder =  'C:/Users/lolol/OneDrive/Документы/Дипломная работа/test'
transform = transforms.Compose([transforms.Resize((32,32)),transforms.ToTensor()])
train_dataset = datasets.ImageFolder(train_folder, transform=transform)
test_dataset = datasets.ImageFolder(test_folder, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Instantiate the neural network and define the loss function and optimizer
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# print(torch.__version__)

# Train the neural network
for epoch in range(10):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Forward pass
        scores = model(data)
        loss = criterion(scores, targets)
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Print training progress
        if batch_idx % 100 == 0:
            print(f'Epoch [{epoch+1}/{10}], Batch [{batch_idx}/{len(train_loader)}], Loss: {loss.item():.4f}')
    
    # Evaluate the neural network on the test set
    with torch.no_grad():
        correct = 0
        total = 0
        for data, targets in test_loader:
            scores = model(data)
            _, predicted = torch.max(scores.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
        accuracy = 100 * correct / total
        print(f'Test Accuracy: {accuracy:.2f}%')

model.save('CNNs_model.h5')
print('Модель создана')

Epoch [1/10], Batch [0/29], Loss: 0.6901
Test Accuracy: 41.67%
Epoch [2/10], Batch [0/29], Loss: 0.6725
Test Accuracy: 45.83%
Epoch [3/10], Batch [0/29], Loss: 0.6167
Test Accuracy: 45.83%
Epoch [4/10], Batch [0/29], Loss: 0.5703
Test Accuracy: 50.00%
Epoch [5/10], Batch [0/29], Loss: 0.5970
Test Accuracy: 45.83%
Epoch [6/10], Batch [0/29], Loss: 0.5487
Test Accuracy: 58.33%
Epoch [7/10], Batch [0/29], Loss: 0.6218
Test Accuracy: 83.33%
Epoch [8/10], Batch [0/29], Loss: 0.4449
Test Accuracy: 66.67%
Epoch [9/10], Batch [0/29], Loss: 0.4702
Test Accuracy: 58.33%
Epoch [10/10], Batch [0/29], Loss: 0.3721
Test Accuracy: 70.83%


AttributeError: 'Net' object has no attribute 'save'

In [ ]:
# Set up the camera feed
cap = cv2.VideoCapture(0)

while True:
    # Capture a frame from the camera feed
    ret, frame = cap.read()
    
    # Convert the frame from BGR to RGB
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Apply the transforms to the frame
    input_img = transform(frame)
    
    # Run the model on the input image
    outputs = model([input_img])
    
    # Get the predicted bounding boxes and labels
    boxes = outputs[0]['boxes'].detach().numpy()
    labels = outputs[0]['labels'].detach().numpy()
    
    # Draw the bounding boxes on the frame
    for i in range(len(boxes)):
        if labels[i] == 1:
            box = boxes[i]
            x1, y1, x2, y2 = box[0], box[1], box[2], box[3]
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
    
    # Show the frame
    cv2.imshow('frame', cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
    
    # Exit on key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera feed and close all windows
cap.release()
cv2.destroyAllWindows()
